In [1]:
import os 

In [2]:
%pwd

'c:\\Users\\admin\\Desktop\\Induction_Furnace_Carbon_Sulphur_Response\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\admin\\Desktop\\Induction_Furnace_Carbon_Sulphur_Response'

In [5]:
# writing entity : is just like a return type 


from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    

In [6]:
from src.Mlflow_Project.constants import *
from src.Mlflow_Project.utils.utility import FileOperations

In [7]:
# configuration manager class , will read all yaml files 

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = FileOperations.read_yaml(config_filepath)
        self.params = FileOperations.read_yaml(params_filepath)
        self.schema = FileOperations.read_yaml(schema_filepath)

        FileOperations.create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        FileOperations.create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file
        )

        return data_ingestion_config

In [8]:
# Updating our components 

import os 
import urllib.request as request
import zipfile 
from src.Mlflow_Project import logger 
from src.Mlflow_Project.utils.utility import FileOperations

In [28]:

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            try:
                filename, headers = request.urlretrieve(self.config.source_URL, self.config.local_data_file)
                print(f"{filename} downloaded successfully with the following info:\n{headers}")
            except Exception as e:
                print(f"An error occurred while downloading the file: {e}")
        else:
            print(f"File already exists at {self.config.local_data_file}")



In [9]:
'''class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {FileOperations.get_size(Path(self.config.local_data_file))}")
            '''

In [29]:
# update the pipeline 

try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
except Exception as e:
    print("An error occurred:", e)
    

[2023-11-07 19:13:47,014: INFO: yaml file: config\config.yaml loaded successfully]
[2023-11-07 19:13:47,018: INFO: yaml file: params.yaml loaded successfully]
[2023-11-07 19:13:47,022: INFO: yaml file: schema.yaml loaded successfully]
[2023-11-07 19:13:47,026: INFO: created directory at: artifacts]
[2023-11-07 19:13:47,030: INFO: created directory at: artifacts/data_ingestion]


artifacts/data_ingestion/Mild_Steel.csv downloaded successfully with the following info:
Connection: close
Content-Length: 872915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: text/plain; charset=utf-8
ETag: "5f9c7d795121097358bf856d8421661a80d1b8af4b2e1905a45e29703076bf22"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 92BA:31952:15CF1D:267517:654A3EE4
Accept-Ranges: bytes
Date: Tue, 07 Nov 2023 13:43:52 GMT
Via: 1.1 varnish
X-Served-By: cache-bom4749-BOM
X-Cache: HIT
X-Cache-Hits: 1
X-Timer: S1699364632.247570,VS0,VE2
Vary: Authorization,Accept-Encoding,Origin
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastly-Request-ID: 1f12d4b70a6c7ca8148c7f19b960b13e7cb13c77
Expires: Tue, 07 Nov 2023 13:48:52 GMT
Source-Age: 48


